In [3]:
#import the dataset and strip off unwanted columns 
import pandas as pd

df = pd.read_csv('US_Accidents_March23.csv', dtype={"fips": str})
df = df[['Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Street', 'City', 'County', 'State', 'Zipcode', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)','Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Junction', 'Stop', 'Traffic_Signal', 'Sunrise_Sunset']]

In [37]:
df[['Zipcode', 'Mailing_Zip']] = df['Zipcode'].str.split('-', expand=True)


Index(['Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Junction',
       'Stop', 'Traffic_Signal', 'Sunrise_Sunset', 'Mailing_Zip'],
      dtype='object')

In [43]:
import numpy as np
county_entry_counts = df.groupby('Zipcode').size().reset_index(name='entry_count')

county_entry_counts['entry_count_scaled'] = np.log10(county_entry_counts['entry_count'])

print(county_entry_counts)


      Zipcode  entry_count  entry_count_scaled
0       01001           16            1.204120
1       01002           23            1.361728
2       01003            1            0.000000
3       01005           18            1.255273
4       01007           25            1.397940
...       ...          ...                 ...
24231   99361            5            0.698970
24232   99362           16            1.204120
24233   99371            3            0.477121
24234   99401            7            0.845098
24235   99403            1            0.000000

[24236 rows x 3 columns]


In [10]:
#import the geojson for US counties 
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [33]:
counties['features'][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

In [45]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'firefox'

fig = px.choropleth_mapbox(
    county_entry_counts,
    geojson=counties,
    locations= county_entry_counts['Zipcode'],  # Replace 'FIPS' with the actual column name in your DataFrame
    color='entry_count_scaled',  # Replace 'Density' with the column you want to visualize
    hover_name='Zipcode',  # Replace 'County' with the column containing county names
    mapbox_style="carto-positron",
    center=dict(lat=37.0902, lon=-95.7129),
    zoom=3,
    opacity=0.7,
    title='Density Car Crash Map of the Continental US'
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title='Density'
        # Adjust tick labels as needed
    )
)

fig.show()

In [9]:
from dash import Dash, html, dash_table, dcc
import plotly.express as px
import pandas as pd